<a href="https://colab.research.google.com/github/dcastf01/aena_scraping_licitaciones/blob/main/Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/dcastf01/aena_scrapping_licitaciones.git

Cloning into 'aena_scrapping_licitaciones'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 132 (delta 70), reused 50 (delta 25), pack-reused 0
Receiving objects: 100% (132/132), 7.68 MiB | 6.93 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [2]:
%cd aena_scrapping_licitaciones

/content/aena_scrapping_licitaciones


In [3]:
#@title Imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [4]:
#@title setup
filename = "clean_data_without_outliers.parquet" #@param ["clean_data_with_outliers.parquet", "clean_data_without_outliers.parquet"]
year_min_to_visualize = 2014 #@param {type:"integer"}
year_max_to_visualize = 2022 #@param {type:"integer"}

assert year_max_to_visualize>=year_min_to_visualize, 'please xlim_max must be greater than xlim_min'
folder_with_data='data'

input_file=os.path.join(folder_with_data,filename)
df=pd.read_parquet(input_file)

df=df[(
    df['Fecha de Contrato'].dt.year>=year_min_to_visualize) 
    & (df['Fecha de Contrato'].dt.year<=year_max_to_visualize)]
print('shape',df.shape)
print('initial colums \n',df.columns)
print('dtypes \n',df.dtypes)

shape (4898, 16)
initial colums 
 Index(['Fecha de Contrato', 'No Expediente', 'Destino', 'Titulo:',
       'Fecha de inicio', 'Fecha fin actual', 'Importe bruto licitacion:',
       'Importe bruto adjudicacion:', 'Forma adjudicacion:',
       'Numero de licitadores:', 'NIF', 'Razon social:', 'Fecha aprobacion',
       'duration_days', 'Importe bruto adjudicacion per day',
       '% bruto respect licitacion'],
      dtype='object')
dtypes 
 Fecha de Contrato                     datetime64[ns]
No Expediente                                 object
Destino                                       object
Titulo:                                       object
Fecha de inicio                       datetime64[ns]
Fecha fin actual                      datetime64[ns]
Importe bruto licitacion:                    float64
Importe bruto adjudicacion:                  float64
Forma adjudicacion:                           object
Numero de licitadores:                       float64
NIF                      

##Insights

### Creating report to have insights

###Main insights from this report
    * We have a Nº Expediente with several repetitions
    * We have a few values in the fields with "Importe" that break all the numeric análisis
    * Elecnor is the company with more "Licitaciones"
    * Exist contracts with 0
    * Contracts where the % bruto respect licitacion has the highest value or lowest value

##main insights from:
https://www.hacienda.gob.es/RSC/OIReScon/informe-anual-supervision-2020/ias-2020.pdf#page=30&zoom=100,101,132

* Cantidad de licitaciones por año
* Duración de licitaciones por año
* Importe bruto por día por año
* % bruto respecto al inicial
* Importe por año
Esas son las principales, luego cambiar la palabra año por aeropuerto / numero de licitaciones / la forma de adjudicación
* Numero de solicitudes de empresa por año
* Número de licitaciones por aeropuerto
* Importe por aeropuerto/departamento

* Company with more contracts




Licitaciones por año

In [18]:
import plotly.express as px
from pandas.api.types import is_datetime64_any_dtype as is_datetime
def generate_grouper(df,column_to_group):
    grouper_column=[]
    for column in column_to_group:
        if  column is None or column not in df.columns:
            pass
        elif is_datetime(df[column]):
            grouper_column.append(df[column].dt.year)
        else:
            grouper_column.append(df[column])
    return grouper_column
def plot_bar_count_column(df,column_to_sum,**kwargs):
    grouper_column=generate_grouper(df,[column_to_sum])
    df_to_plot=df.groupby(by=[*grouper_column])[column_to_sum].count().rename('count',axis=1)
    fig= px.bar(df_to_plot,text_auto='.2s')
    return fig

def plot_bar_sum_column(df,column_to_sum,x=None,
                        color=None,variable_to_sort='columns_to_sum',
                        is_ascending_sort_by_color=False,text_auto='.2s'):
    
    grouper_column=generate_grouper(df,[x,color])
    df_to_plot=df.groupby(by=[*grouper_column])[column_to_sum].sum().reset_index()
    )
    #solucionar lo de la parte del sort mientras se hace plot ya que no se me ordena como quiero, quizá se hace en el plot
    if variable_to_sort=='columns_to_sum':
        column_to_sort=columns_to_sum
    elif variable_to_sort=='choosing_dimension_to_colorbar':
        column_to_sort=color

    if color:
        df_to_plot.sort_values(column_to_sum,inplace=True,ascending=is_ascending_sort_by_color)
    print(df_to_plot.head()
    # return df_to_plot
    fig= px.bar(df_to_plot,x=x,y=column_to_sum,color=color,text_auto='.2s')
    return fig

In [20]:
#@title variables to print
kind_of_plot = "bar_count" #@param ["bar_count","bar_sum"]
axis_x= "Fecha de Contrato" #@param ["Fecha de Contrato", "Destino","Forma adjudicacion"]
columns_to_sum= "Importe bruto licitacion:" #@param ['duration_days','Importe bruto licitacion:']
choosing_dimension_to_colorbar = "Numero de licitadores:" #@param ["None", "Destino", "Numero de licitadores:","Forma adjudicacion"]
sort_by_column_to_sum_or_by_dimension_to_colorbar="choosing_dimension_to_colorbar" #@param ['columns_to_sum','choosing_dimension_to_colorbar']
is_ascending_sort_by_color = True #@param {type:"boolean"}

if choosing_dimension_to_colorbar=='None':
    choosing_dimension_to_colorbar=None


if kind_of_plot =='bar_count':
    func=plot_bar_count_column
    
elif kind_of_plot=='bar_sum':
    func=plot_bar_sum_column


plot_bar_sum_column(df,
                    columns_to_sum,
                    axis_x,
                    choosing_dimension_to_colorbar,
                    is_ascending_sort_by_color
                    ).show()


   Fecha de Contrato  Numero de licitadores:  Importe bruto licitacion:
0               2014                     1.0                 5548377.45
1               2014                     2.0                 2991281.99
2               2014                     3.0                 2491344.90
3               2014                     4.0                 1220565.67
4               2014                     5.0                  121000.00


In [21]:
columns_to_sum

'Importe bruto licitacion:'

In [116]:
fig.show()

### contracts with import 0

In [ ]:
df[df['Importe bruto adjudicación(€)']==0]

,Fecha de Contrato,Nº Expediente,Destino,Importe bruto adjudicación(€),Importe neto adjudicación(€),Valor estimado contrato neto(€),Título:,Fecha de inicio,Fecha fin actual,Importe bruto licitación:,...,Importe neto adjudicación:,Valor estimado contrato neto:,Forma adjudicación:,Número de licitadores:,Incidencias del contrato:,NIF,Razón social:,Unnamed: 26,duration_days,Valor estimado contrato neto(€)_per_day
0,NaT,DPM-280/2014-2,VARIOS AEROPUERTOS,0.0,0.0,4000000.0,A.M. PROYECTO Y EJECUCIÓN AISLAMIENTO ACÚSTICO...,NaT,2018-03-26,4840000.0,...,0.0,4000000.0,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,9,0,NaN,NaN,NaN,NaN,NaN
121,2022-05-12,DEA-299/2021-2,APTO.ZARAGOZA,0.0,0.0,0.0,PRESTACIÓN A TERCEROS DEL SERVICIO DE ASISTENC...,2022-06-12,2029-06-12,0.0,...,0.0,0.0,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,2,0,B083840900,SERV LOG COMB AVIACION SL,NaN,2557.0,0.0
441,2022-01-12,DTC-112/2021-110,VARIOS AEROPUERTOS,0.0,0.0,0.0,ACUERDO MARCO PARA EL SUMINISTRO E INSTALACIÓN...,2022-01-12,NaT,0.0,...,0.0,0.0,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,10,0,NaN,NaN,NaN,NaN,NaN
511,2021-12-14,DEF-98/2021-1,DIR.ECONOMICO-FINANCIERA,0.0,0.0,0.0,SERVICIO DE MEDIACIÓN DE SEGUROS PRIVADOS DE A...,2022-01-14,2024-01-14,0.0,...,0.0,0.0,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,4,0,A028109247,AON IBERIA CORREDURIA SEG,NaN,730.0,0.0
520,2021-12-13,DEF-98/2021-2,DIR.ECONOMICO-FINANCIERA,0.0,0.0,0.0,SERVICIO DE MEDIACIÓN DE SEGUROS PRIVADOS DE A...,2022-01-13,2024-01-13,0.0,...,0.0,0.0,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,3,0,A081332322,MARSH SA,NaN,730.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9043,2015-01-23,DIN-192/2014-12,D PLANIF AER CONTR REGULA,0.0,0.0,0.0,ACUERDO MARCO. SUMINISTRO Y/O INSTALACIÓN SIST...,2015-02-09,2018-02-09,0.0,...,0.0,0.0,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,1,0,A028002335,"SICE, SDAD IBER. CONST.",NaN,1096.0,0.0
9044,2015-01-23,DIN-192/2014-15,D PLANIF AER CONTR REGULA,0.0,0.0,0.0,ACUERDO MARCO. SUMINISTRO Y/O INSTALACIÓN SIST...,2015-02-09,2018-02-09,0.0,...,0.0,0.0,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,1,0,U087191235,UTE SERVEO INF-SERVEO SE,NaN,1096.0,0.0
9045,2015-01-23,DIN-192/2014-16,D PLANIF AER CONTR REGULA,0.0,0.0,0.0,ACUERDO MARCO. SUMINISTRO Y/O INSTALACIÓN SIST...,2015-02-09,2018-02-09,0.0,...,0.0,0.0,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,1,0,U087196838,UTE IMESAPI SA Y NORIN SL,NaN,1096.0,0.0
9122,2014-10-16,DEA-144/2014-12,VARIOS AEROPUERTOS,0.0,0.0,0.0,ACUERDO MARCO 2014-2017 PARA LA ADQUISICIÓN DE...,2014-10-16,2018-10-16,0.0,...,0.0,0.0,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,1,0,A060622743,NISSAN IBERIA S.A,NaN,1461.0,0.0


###Checking what happen with the "N* Expediente" most repeated

In [ ]:
df[df['Nº Expediente']=='DTC-83/2015-1']

,Fecha de Contrato,Nº Expediente,Destino,Importe bruto adjudicación(€),Importe neto adjudicación(€),Valor estimado contrato neto(€),Título:,Fecha de inicio,Fecha fin actual,Importe bruto licitación:,...,Importe neto adjudicación:,Valor estimado contrato neto:,Forma adjudicación:,Número de licitadores:,Incidencias del contrato:,NIF,Razón social:,Unnamed: 26,duration_days,Valor estimado contrato neto(€)_per_day
8537,2015-01-10,DTC-83/2015-1,VARIOS AEROPUERTOS,3690496.08,3049996.76,5336472.99,SERVICIOS DE TELECOMUNICACIONES 2016-2019. LOT...,2015-01-10,2019-01-10,4304754.88,...,3049996.76,5336472.99,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,2,0,A082018474,TELEFONICA DE ESPAÑA SAU,NaN,1461.0,3652.616694
8539,2015-01-10,DTC-83/2015-1,APTO.A.SUAREZ MAD-BARAJAS,747705.36,617938.31,1090994.37,MAD.-SERVICIOS DE TELECOMUNICACIONES 2016-2019...,2015-01-10,2022-10-28,880068.79,...,617938.31,1090994.37,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,2,Tipo incidencia\r\n\t\t\t\t\t\t\t\t\t\t\t\nDes...,A082018474,TELEFONICA DE ESPAÑA SAU,NaN,2848.0,383.073866
8543,2015-01-10,DTC-83/2015-1,APTO.A.SUAREZ MAD-BARAJAS,277310.43,229182.17,393860.37,MAD.-SERVICIOS DE TELECOMUNICACIONES 2016-2019...,2015-01-10,2022-10-28,317714.03,...,229182.17,393860.37,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,2,Tipo incidencia\r\n\t\t\t\t\t\t\t\t\t\t\t\nDes...,A082018474,TELEFONICA DE ESPAÑA SAU,NaN,2848.0,138.293669
8544,2015-01-10,DTC-83/2015-1,APTO.J.TARRAD BARCELONA P,202671.65,167497.23,210419.43,BCN.-SERVICIOS DE TELECOMUNICACIONES 2016-2019...,2015-01-10,2022-10-28,169738.34,...,167497.23,210419.43,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,2,Tipo incidencia\r\n\t\t\t\t\t\t\t\t\t\t\t\nDes...,A082018474,TELEFONICA DE ESPAÑA SAU,NaN,2848.0,73.883227
8545,2015-01-10,DTC-83/2015-1,APTO.J.TARRAD BARCELONA P,170210.12,140669.52,185172.74,BCN.-SERVICIOS DE TELECOMUNICACIONES 2016-2019...,2015-01-10,2022-10-28,149372.67,...,140669.52,185172.74,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,2,Tipo incidencia\r\n\t\t\t\t\t\t\t\t\t\t\t\nDes...,A082018474,TELEFONICA DE ESPAÑA SAU,NaN,2848.0,65.018518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8635,2015-01-10,DTC-83/2015-1,HLPTO.ALGECIRAS,5601.43,4629.28,6870.27,AEI.-SERVICIOS DE TELECOMUNICACIONES 2016-2019...,2015-01-10,2022-10-28,5542.02,...,4629.28,6870.27,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,2,Tipo incidencia\r\n\t\t\t\t\t\t\t\t\t\t\t\nDes...,A082018474,TELEFONICA DE ESPAÑA SAU,NaN,2848.0,2.412314
8636,2015-01-10,DTC-83/2015-1,APTO.BURGOS,4748.22,3924.15,6921.95,RGS.-SERVICIOS DE TELECOMUNICACIONES 2016-2019...,2015-01-10,2022-10-28,5583.70,...,3924.15,6921.95,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,2,Tipo incidencia\r\n\t\t\t\t\t\t\t\t\t\t\t\nDes...,A082018474,TELEFONICA DE ESPAÑA SAU,NaN,2848.0,2.430460
8637,2015-01-10,DTC-83/2015-1,APTO.ALBACETE,4716.69,3898.09,7018.77,ABC.-SERVICIOS DE TELECOMUNICACIONES 2016-2019...,2015-01-10,2022-10-28,5661.81,...,3898.09,7018.77,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,2,Tipo incidencia\r\n\t\t\t\t\t\t\t\t\t\t\t\nDes...,A082018474,TELEFONICA DE ESPAÑA SAU,NaN,2848.0,2.464456
8638,2015-01-10,DTC-83/2015-1,HLPTO.CEUTA,3876.47,3524.06,6413.60,JCU.-SERVICIOS DE TELECOMUNICACIONES 2016-2019...,2015-01-10,2022-10-28,4703.30,...,3524.06,6413.60,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,2,Tipo incidencia\r\n\t\t\t\t\t\t\t\t\t\t\t\nDes...,A082018474,TELEFONICA DE ESPAÑA SAU,NaN,2848.0,2.251966


### We have a value anomalous in relacion "importes" 


In [ ]:
df.sort_values('Importe neto adjudicación(€)',ascending=False) 

,Fecha de Contrato,Nº Expediente,Destino,Importe bruto adjudicación(€),Importe neto adjudicación(€),Valor estimado contrato neto(€),Título:,Fecha de inicio,Fecha fin actual,Importe bruto licitación:,...,Importe neto adjudicación:,Valor estimado contrato neto:,Forma adjudicación:,Número de licitadores:,Incidencias del contrato:,NIF,Razón social:,Unnamed: 26,duration_days,Valor estimado contrato neto(€)_per_day
478,2021-12-28,DEA-384/2021,VARIOS AEROPUERTOS,7.274669e+08,6.012123e+08,6.012123e+08,SERVICIOS DE NAVEGACION AEREA EN AEROPUERTOS D...,2022-01-01,2027-01-01,7.274669e+08,...,6.012123e+08,6.012123e+08,PROCEDIMIENTO NEGOCIADO SIN PUBLICIDAD,1,0,Q02822001J,ENAIRE,NaN,1826.0,329250.989535
4499,2018-12-19,SEG-368/2018-100,VARIOS AEROPUERTOS,4.598558e+08,3.800461e+08,3.800461e+08,ACUERDO MARCO PARA EL SUMINISTRO E INSTALACIÓN...,2018-12-19,2023-12-19,4.598558e+08,...,3.800461e+08,3.800461e+08,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,5,0,A028166007,TECOSA,NIF:\r\n\t\t\t\t\t\t\t\t\t\t U088224654 ...,1826.0,208130.394304
9390,2003-04-07,DPB-823/2002-1,APTO.A.SUAREZ MAD-BARAJAS,3.738636e+08,3.222962e+08,NaN,SUMINISTRO DE ENERGÍA ELÉCTRICA Y TERMICA PARA...,2006-12-18,2022-09-18,3.738636e+08,...,3.222962e+08,NaN,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,7,Tipo incidencia\r\n\t\t\t\t\t\t\t\t\t\t\t\nDes...,A007088206,SAMPOL INGENIER Y OBRA SA,NaN,5753.0,NaN
72,2022-05-25,DIN-492/2019,APTO.PALMA DE MALLORCA,2.401397e+08,1.984625e+08,2.239225e+08,REMODELACIÓN DEL ÁREA TERMINAL DEL AEROPUERTO ...,2022-06-25,2025-09-25,2.709463e+08,...,1.984625e+08,2.239225e+08,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,2,0,A081638108,ACCIONA CONSTRUCCION SA,NaN,1188.0,188486.979798
6889,2017-04-21,MAD-584/2016,APTO.A.SUAREZ MAD-BARAJAS,1.704890e+08,1.409000e+08,2.115640e+08,SERVICIO DE OPERACIÓN Y MANTENIMIENTO DE SATE ...,2017-01-07,2022-01-07,1.708176e+08,...,1.409000e+08,2.115640e+08,PROCEDIMIENTO DE LICITACIÓN CON NEGOCIACIÓN,1,Tipo incidencia\r\n\t\t\t\t\t\t\t\t\t\t\t\nDes...,B086777463,SIEMENS LOGISTICS SLU,NaN,1826.0,115862.001232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,2021-06-22,SGC-28/2021-134,VARIOS AEROPUERTOS,0.000000e+00,0.000000e+00,NaN,ACUERDO MARCO PARA LA CONTRATACIÓN DE LOS SERV...,2021-09-07,2025-09-07,0.000000e+00,...,0.000000e+00,NaN,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,1,0,028852619Q,MARIN BENITEZ RAFAEL,NaN,1461.0,NaN
1195,2021-06-22,SGC-28/2021-136,VARIOS AEROPUERTOS,0.000000e+00,0.000000e+00,NaN,ACUERDO MARCO PARA LA CONTRATACIÓN DE LOS SERV...,2021-09-07,2025-09-07,0.000000e+00,...,0.000000e+00,NaN,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,1,0,B088581871,LJ BUDDHI QONSOLIDA CONSL,NaN,1461.0,NaN
1196,2021-06-22,SGC-28/2021-138,VARIOS AEROPUERTOS,0.000000e+00,0.000000e+00,NaN,ACUERDO MARCO PARA LA CONTRATACIÓN DE LOS SERV...,2021-09-07,2025-09-07,0.000000e+00,...,0.000000e+00,NaN,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,1,0,B085449528,MARIA DEL MAR DE VILLA MO,NaN,1461.0,NaN
1197,2021-06-22,SGC-28/2021-139,VARIOS AEROPUERTOS,0.000000e+00,0.000000e+00,NaN,ACUERDO MARCO PARA LA CONTRATACIÓN DE LOS SERV...,2021-09-07,2025-09-07,0.000000e+00,...,0.000000e+00,NaN,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,1,0,006993004S,FERNANDEZ DE LAS HERAS JE,NaN,1461.0,NaN


Before to check the outliers for us is interest know what are the contract where % bruto respect licitación was highest or lowest. We only are interested in know the date where and to who

In [ ]:
display('Highest',
      df[['Fecha de Contrato','Destino','Razón social:','% bruto respect licitacion']
         ].sort_values('% bruto respect licitacion',ascending=False).head(10)
      )
display('Lowest',
      df[['Fecha de Contrato','Destino','Razón social:','% bruto respect licitacion']
         ].sort_values('% bruto respect licitacion').head(10)
      )

'Highest'

,Fecha de Contrato,Destino,Razón social:,% bruto respect licitacion
8549,2015-10-08,APTO.SON BONET,VODAFONE ESPAÑA SA,4.938457
8529,2015-10-08,APTO.ZARAGOZA,VODAFONE ESPAÑA SA,3.609487
6779,2017-06-19,VARIOS AEROPUERTOS,BOSLAN INGEN. Y CONSULTOR,2.660357
8530,2015-10-08,APTO.MENORCA,VODAFONE ESPAÑA SA,2.385681
4474,2019-01-23,DIR.OPERACION Y RED APTOS,TERMINALES CANARIOS SL,2.000000
8559,2015-10-08,APTO.LA GOMERA,VODAFONE ESPAÑA SA,1.848624
8566,2015-10-08,APTO.HUESCA-PIRINEOS,VODAFONE ESPAÑA SA,1.625640
8633,2015-10-01,APTO.VITORIA,TELEFONICA DE ESPAÑA SAU,1.510805
8662,2015-10-01,APTO.CÓRDOBA,TELEFONICA DE ESPAÑA SAU,1.449733
8567,2015-10-08,HLPTO.CEUTA,VODAFONE ESPAÑA SA,1.418788


'Lowest'

,Fecha de Contrato,Destino,Razón social:,% bruto respect licitacion
0,NaT,VARIOS AEROPUERTOS,NaN,0.000000
9055,2015-02-09,APTO.PALMA DE MALLORCA,TECNIKRONOS SL,0.000000
3899,2019-07-17,DIR.ECONOMICO-FINANCIERA,DEUDAE COMPENS DEUDAS SL,0.000000
4261,2019-03-28,DIR.OPERACION Y RED APTOS,"EXOLUM AVIATION, S.A.U.",0.008264
4365,2019-02-22,APTO.ASTURIAS,"EXOLUM AVIATION, S.A.U.",0.033333
4366,2019-02-22,APTO.JEREZ,"EXOLUM AVIATION, S.A.U.",0.041667
4364,2019-02-22,APTO.A CORUÑA,"EXOLUM AVIATION, S.A.U.",0.041667
4368,2019-02-22,APTO.REUS,"EXOLUM AVIATION, S.A.U.",0.055556
9070,2015-02-03,APTO.A.SUAREZ MAD-BARAJAS,"SERVEO SERVICIOS, SAU",0.113822
2032,2020-12-21,VARIOS AEROPUERTOS,UTE TRABLISA-VISABREN,0.131659


###create report without outlier

In [ ]:
print('shape before remove outlier', df.shape)
df_wihtout_outliers=df[df['Importe bruto adjudicación(€)']<=1e+06]
print('shape after remove outlier', df_wihtout_outliers.shape)

shape before remove outlier (9391, 21)
shape after remove outlier (8112, 21)


In [ ]:
report=create_report(df_wihtout_outliers.drop(['Importe bruto licitación:','duration_days'],axis=1))
report.save('without_outliers.html')

E-1019 (DUPLICATE_FACTORS): FactorRange must specify a unique list of categorical factors for an axis: duplicate factors found: 'Importe neto ...ón:'


Report has been saved to without_outliers.html!
